In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/geiot

/content/drive/MyDrive/Colab Notebooks/geiot


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from tqdm import tqdm
from torchvision import models,transforms
from PIL import Image
import cv2
import sys
import json
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import os
import os.path as osp
from glob import glob
import pandas as pd
import shutil

In [2]:
torch.__version__

'1.9.0'

In [3]:
torchvision.__version__

'0.10.0'

In [4]:
"hjsdgjd jsajfkj".split(" ")

['hjsdgjd', 'jsajfkj']

# データのフォルダーを分ける

Root----------input<br>
｜&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;｜<br>
｜&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;｜ー&nbsp;train&nbsp;ーimages<br>
｜&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;｜ー&nbsp;test&nbsp;ーimages<br>
｜<br>
&nbsp;&nbsp;ーorignal_data(GEBKI,MTFL)


In [5]:
Root="TeamB"
input_dir=osp.join(Root,"input")
os.makedirs(input_dir,exist_ok=True)
train_dir=osp.join(input_dir,"train")
os.makedirs(train_dir,exist_ok=True)
test_dir=osp.join(input_dir,"test")
os.makedirs(test_dir,exist_ok=True)

## GENKIデータセットのフォルダ分け

In [6]:
GENKI_imgdata_dir="TeamB/GENKI-R2009a/GENKI-R2009a/files"
GENKI_imgtxt_path="TeamB/GENKI-R2009a/GENKI-R2009a/Subsets/GENKI-4K/GENKI-4K_Images.txt"
GENKI_Labeltxt_path="TeamB/GENKI-R2009a/GENKI-R2009a/Subsets/GENKI-4K/GENKI-4K_Labels.txt"

#labelデータ取得
print(osp.exists(GENKI_Labeltxt_path))
with open(GENKI_Labeltxt_path,mode="r") as f:
    labeltext=f.readlines()
label_col=["Label","_","__","___"]
print(labeltext)
labeltext=[i.replace( '\n' , '' ).split(" ") for i in labeltext]
labeldata=pd.DataFrame(data=labeltext,columns=label_col)

#imagepathデータ取得
with open(GENKI_imgtxt_path,mode="r") as f:
    image_paths=f.readlines()

image_paths=[osp.join(GENKI_imgdata_dir,i.replace( '\n' , '' )) for i in image_paths]
print(image_paths)
pathdata=pd.DataFrame(image_paths,columns=["imagePath"])

#データをまとめる
GENKI_data=pd.concat([pathdata,labeldata["Label"]],axis=1)

#データ分割
train_data,test_data=train_test_split(GENKI_data,test_size=int(len(GENKI_data)/5))

#フォルダ分け
for train_path in train_data.imagePath:
    shutil.copy(train_path,osp.join(train_dir,osp.basename(train_path)))
    
for test_path in test_data.imagePath:
    shutil.copy(test_path,osp.join(test_dir,osp.basename(test_path)))

train_data.to_csv(osp.join(train_dir,"train_anno(GENKI).csv"))
test_data.to_csv(osp.join(test_dir,"test_anno(GENKI).csv"))

"""
#print(image_paths)
image_label_dict={"imagePath":[],"Label":[]}
for path in image_paths:
  #basenaemは拡張子を含んだファイル名,splitextは「.」で分割するので[0]を指定すると、拡張子なしのファイル名が手に入る
  label= osp.splitext(os.path.basename(path))[0]
  if not ((label[4:6]) in ["AF","AN","DI","HA","NE","SA","SU"]):
    continue
  image_label_dict["Label"].append(label[4:6])
  image_label_dict["imagePath"].append(path)
"""

True
['1 -0.021162 0.059530 -0.039662\n', '1 -0.057745 0.083098 -0.094952\n', '1 0.095993 0.028798 0.065996\n', '1 0.000000 0.047124 0.171268\n', '1 0.036073 0.043633 -0.181721\n', '1 0.004091 0.171042 -0.009009\n', '1 0.034871 -0.006981 0.077530\n', '1 -0.027925 0.064577 0.183285\n', '1 -0.200981 0.096750 -0.033657\n', '1 0.127409 0.017453 0.138530\n', '1 0.005282 0.270526 0.043451\n', '1 0.166094 0.125847 0.058592\n', '1 -0.194974 0.097738 0.145126\n', '1 -0.045379 0.043633 -0.031309\n', '1 -0.157080 -0.031416 0.153307\n', '1 0.358359 0.069813 0.017005\n', '1 -0.193732 -0.066323 -0.025968\n', '1 -0.144862 -0.001745 0.062589\n', '1 0.116937 0.129154 -0.304920\n', '1 0.009280 0.204204 -0.063831\n', '1 0.373500 0.020944 0.139925\n', '1 -0.003598 0.205949 0.016338\n', '1 0.282743 -0.054443 -0.084081\n', '1 -0.116937 -0.064577 -0.134023\n', '1 -0.008727 0.020944 0.005747\n', '1 0.518966 0.004364 0.078172\n', '1 -0.025801 -0.251327 0.018646\n', '1 -0.075049 0.040143 0.000000\n', '1 -0.0633

'\n#print(image_paths)\nimage_label_dict={"imagePath":[],"Label":[]}\nfor path in image_paths:\n  #basenaemは拡張子を含んだファイル名,splitextは「.」で分割するので[0]を指定すると、拡張子なしのファイル名が手に入る\n  label= osp.splitext(os.path.basename(path))[0]\n  if not ((label[4:6]) in ["AF","AN","DI","HA","NE","SA","SU"]):\n    continue\n  image_label_dict["Label"].append(label[4:6])\n  image_label_dict["imagePath"].append(path)\n'

In [7]:
train_data

,imagePath,Label
1316,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
55,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
1247,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
186,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
516,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
...,...,...
412,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
2226,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,0
3545,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,0
3222,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,0


In [8]:
test_data

,imagePath,Label
1508,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
2114,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
2625,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,0
2856,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,0
1373,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
...,...,...
3585,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,0
1489,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1
3978,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,0
664,TeamB/GENKI-R2009a/GENKI-R2009a/files\file0000...,1


## MTFLデータセットのフォルダ分け

<h1>textファイルの編集が必要！！！！</h1>
1.最初の行に空欄<br>
2.トレインテキストのみバックスラッシュから/に変更

In [ ]:
train_data.loc[0]["imagePath"]

'/content/drive/MyDrive/Colab Notebooks/geiot/GENKI-R2009a/files/file0000000000000001.jpg'

In [9]:
MTFL_imgdata_dir="TeamB/MTFL"
MTFL_testtxt_path="TeamB/MTFL/testing.txt"
MTFL_traintxt_path="TeamB/MTFL/training.txt"

#testデータ取得
with open(MTFL_testtxt_path,mode="r") as f:
    test_text=f.readlines()
label_col=["_","imagePath","__","___","____","_____","______","_______","________","_________","__________","___________","____________","Label","*","**"]
test_text=[i.split(" ") for i in test_text]
#print(test_text)
testdata=pd.DataFrame(data=test_text,columns=label_col)
testdata=testdata[["imagePath","Label"]]
testdata.imagePath=testdata.imagePath.map(lambda x: osp.join(MTFL_imgdata_dir,x))
testdata=testdata[:-1]
#trainデータ取得
with open(MTFL_traintxt_path,mode="r") as f:
    train_text=f.readlines()
train_text=[i.split(" ") for i in train_text]
traindata=pd.DataFrame(train_text,columns=label_col)
traindata=traindata[["imagePath","Label"]]
print(train_text)
print(traindata)
traindata.imagePath=traindata.imagePath.map(lambda x: osp.join(MTFL_imgdata_dir,x))
traindata=traindata[:-1]

#フォルダ分け

for train_path in traindata.imagePath:
    shutil.copy(train_path,osp.join(train_dir,osp.basename(train_path)))

for test_path in testdata.imagePath:
    shutil.copy(test_path,osp.join(test_dir,osp.basename(test_path)))

traindata.to_csv(osp.join(train_dir,"train_anno(MTFL).csv"))
testdata.to_csv(osp.join(test_dir,"test_anno(MTFL).csv"))

[['', 'lfw_5590/Aaron_Eckhart_0001.jpg', '107.250000', '147.750000', '126.250000', '106.250000', '140.750000', '108.750000', '113.250000', '143.750000', '158.750000', '162.750000', '1', '2', '2', '3\n'], ['', 'lfw_5590/Aaron_Guiel_0001.jpg', '101.250000', '146.750000', '125.250000', '93.750000', '139.750000', '112.250000', '117.750000', '137.750000', '160.750000', '164.750000', '1', '1', '2', '3\n'], ['', 'lfw_5590/Aaron_Peirsol_0001.jpg', '107.750000', '147.750000', '130.250000', '109.250000', '147.750000', '114.250000', '114.250000', '140.750000', '154.250000', '153.750000', '1', '1', '2', '3\n'], ['', 'lfw_5590/Aaron_Pena_0001.jpg', '102.750000', '146.250000', '126.250000', '120.750000', '147.250000', '117.750000', '104.750000', '137.750000', '164.750000', '156.750000', '1', '2', '2', '3\n'], ['', 'lfw_5590/Aaron_Sorkin_0001.jpg', '102.250000', '144.750000', '130.750000', '100.250000', '143.250000', '113.250000', '112.750000', '138.750000', '156.250000', '155.750000', '1', '1', '2',

In [ ]:
testdata

,imagePath,Label
0,/content/drive/MyDrive/Colab Notebooks/geiot/M...,1
1,/content/drive/MyDrive/Colab Notebooks/geiot/M...,1
2,/content/drive/MyDrive/Colab Notebooks/geiot/M...,1
3,/content/drive/MyDrive/Colab Notebooks/geiot/M...,2
4,/content/drive/MyDrive/Colab Notebooks/geiot/M...,1
...,...,...
2990,/content/drive/MyDrive/Colab Notebooks/geiot/M...,2
2991,/content/drive/MyDrive/Colab Notebooks/geiot/M...,2
2992,/content/drive/MyDrive/Colab Notebooks/geiot/M...,1
2993,/content/drive/MyDrive/Colab Notebooks/geiot/M...,2


In [ ]:
a=[1,2,3]
b=[4,5]

In [ ]:
for i,j in zip(a,b):
  print(i,j)

1 4
2 5
